In [ ]:
#Author: Rong Li

In [ ]:
%%capture output
!pip install spotipy

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
drive.mount('/content/drive')
os.chdir("drive/MyDrive/YOUR PATH/")#change it to your path
#NOTE: !cd does not work in colab

In [ ]:
# SET UP YOUR INFORMATION
token = "ADD YOUR REQUESTED TOKEN HERE"
client_credentials_manager = SpotifyClientCredentials(client_id="ADD YOUR CLIENT_ID HERE", client_secret="ADD YOUR CLIENT_SECRET HERE")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager,auth = token)

In [ ]:
# USERNAMES YOU WANT TO SCRAPE, SAVED IN ONE COLUMN CALLED "Usernames" IN A .CSV
user = pd.read_csv('YOUR FILE NAME.csv')
user.head()

In [ ]:
user_list = list(user.Usernames)
columns = ['username','playlist_uri','track_uri','track_name',
          'artist_uri','artist_name','album','track_pop','acousticness',
          'danceability','duration_ms','energy','instrumentalness',
          'key','liveness','loudness','mode','speechiness',
          'tempo','time_signature','valence']
dataset = pd.DataFrame(columns=columns)

In [ ]:
def get_info(start_user_idx, end_user_idx,user_list,dataset):
  for i in range(start_user_idx, end_user_idx+1):
    try:
      playlists = sp.user_playlists(user_list[i])["items"]#the playlist of the ith user
    except:
      print(f"There is no username: {user_list[i]}")
      continue
    n_playlists = len(playlists)
    if n_playlists == 0:
      continue

    for j in range(n_playlists):
      playlist_URI = playlists[j]["uri"].split(":")[-1]#one of the playlists of the user
      tracks = sp.playlist_tracks(playlist_URI)["items"]
      n_tracks = len(tracks)#number of tracks under this playlist
      try:
        track_uris = [track["track"]["uri"] for track in tracks]

        track_names = [track["track"]["name"] for track in tracks]

        #Artist
        artist_uris = [track["track"]["artists"][0]["uri"] for track in tracks]
        artists_name = [track["track"]["artists"][0]["name"] for track in tracks]

        #album
        albums = [track["track"]["album"]["name"] for track in tracks]

        #Popularity of the track
        tracks_pop = [track["track"]["popularity"] for track in tracks]

        #use track uris to get the features of each track under one playlist
        features_list = ['acousticness','danceability','duration_ms','energy',
                        'instrumentalness','key','liveness','loudness','mode',
                        'speechiness','tempo','time_signature','valence']
        features = []
        for track_uri in track_uris:
          audio_features = sp.audio_features(track_uri)[0]
          #[0]represent audio feature,so you can also loop other features
          track_features = []
          for feature in features_list:
            track_features.append(audio_features[feature])
          features.append(track_features)
        acousticness, danceability,duration_ms,energy,instrumentalness,key,\
        liveness,loudness,mode,speechiness,tempo,time_signature,valence = np.reshape(features,(-1,len(features_list))).T

        values_list = [[user_list[i]]*n_tracks,[playlist_URI]*n_tracks,track_uris,
                      track_names,artist_uris,artists_name,albums,tracks_pop,
                      acousticness, danceability,duration_ms,energy,instrumentalness,
                      key,liveness,loudness,mode,speechiness,tempo,time_signature,valence]

        df_add = pd.DataFrame({m:n for m,n in zip(columns,values_list)})
        dataset = pd.concat([dataset,df_add])
      except:
        continue
  return dataset

In [ ]:
# PICK NO MATTER HOW MANY USERS YOU WANT TO SCRAPE, JUST PROVIDE THEIR INDEX IN THE LIST,
# HOWEVER IF THERE WILL BE A LOT OF USERS, I RECOMMEND RUN CODE IN SEVERAL BATCHES DUE TO THE
# LIMITED SCRAPING RATE SET BY SPOTIFY :)
dataset = get_info(start_user_idx, end_user_idx,user_list,dataset)

In [ ]:
dataset.to_csv('dataset.csv')#save the data! CONGRATULATIONS!!!